In [1]:
import pandas as pd
import numpy as np
import csv
import re
import string
from itertools import product

In [2]:
plate_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate1.csv"
plate_1_repeat = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate1_repeat.csv"
plate_2_1 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_1.csv"
plate_2_2 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_2.csv"
plate_2_3 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_3.csv"
plate_2_repeat = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_repeat.csv"
plate_2_repeat_96 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\plate2_repeat _96.csv"
list_A = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listA.csv"
list_A_repeat = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listA_repeat.csv"
list_B = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listB.csv"
list_B_repeat_end = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listB_repeat _end.csv"
list_B_repeat_96 = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listB_repeat _96.csv"
list_C = "C:\\Users\\Bartek\\OneDrive\\Documents\\Programming\\Python\\Test data\\listC.csv"

In [15]:
class DataError(Exception):
    pass
    
class WellNumberError(Exception):
    pass
    
plate_dimensions = {'96':(8, 12), '384':(16, 24)}
    
class FA:
    """
    :param csv_file: Raw data file in csv format.
    :param type: str
    :param read_in_info: Information needed to pre-process the data.
    :param type: Varies with the data format. Check the appropriate read in function.
    :param g_factor: G-factor.
    :param type: float
    :param data_format: Format in which the raw data was exported (plate1, plate2, listA, listB or listC).
    :param type: str 
    :param wells: Number of wells on the plate.
    :param type: int   """
    
    def __init__(self, data_dict, g_factor):
        self.g_factor = g_factor
        self.data_dict = data_dict
    
    
    @classmethod
    def read_in_envision(cls, csv_file, data_type='plate', wells=384):
        
        #row_letters = list(string.ascii_uppercase)[0:plate_dimensions[str(wells)][0]]   # generate a list of letters for well IDs
        #col_numbers = list(np.arange(1, plate_dimensions[str(wells)][1]+1).astype(str))   # generate a list of numbers for well IDs
        #well_ids = ['%s%s' % (item[0], item[1]) for item in product(row_letters, col_numbers)]   # generate a list of well IDs for the data table

        # ensure the correct number of wells was passed
        if str(wells) not in plate_dimensions:
            raise WellNumberError('Invalid number of wells on the plate, should be 384 or 96.')
        
        if data_type == 'plate':
            try:
                init_data = FA.read_in_plate(csv_file, wells)
                return cls(init_data[0], init_data[1])
            except (ValueError, IndexError):
                raise DataError('incorrect data format or file has no data or incorrect well number')
            except UnboundLocalError:
                raise DataError('File has no data')
        
        if data_type == 'list':
            try:
                init_data = FA.read_in_list(csv_file, wells)
                return cls(init_data[0], init_data[1])
            except (UnboundLocalError, IndexError):
                raise DataError('data format incorrect or file has no data')
                
                
    def read_in_plate(csv_file, wells):

        """ Iterates over the raw data file to find the line numbers at which the metadata table and raw data table begin for 
        each channel and reapeat. Calculates the length of those tables. Finds the G-factor. Determines the format of data (plate 1 or 2).

        :param csv_file: Raw data file in csv format.
        :param type: str
        :param wells: Number of wells on the plate.
        :param type: int
        :return: read_in_info is a list of tuples (one tuple for each channel) with parameters needed for pandas to read in the csv file. """
        
        with open(csv_file) as file:
            all_data_lines = list(csv.reader(file, delimiter=','))   # read the csv file and cast it into a list containing all lines

        blank_indices = list(index for index, item in enumerate(all_data_lines) if item == [])   # list containing indices of all blank rows
        blanks = np.array(blank_indices)   # convert the list of blank indices to a numpy array
        read_in_info = []   # list to store the tuples with parameters needed for pandas to read in the csv file

        for index, item in enumerate(all_data_lines):   # iterate over all lines in the csv file
            if item != [] and re.findall(r"Plate information", item[0]) == ['Plate information'] and re.search(r'Results for', all_data_lines[index + 9][0]) == None and re.findall(r"Formula", all_data_lines[index+1][10]) != ['Formula']:
                skiprows = index + 9   # Set the skiprows parameter for raw data table
                skiprows_meta = index + 1   # Set the skiprows parameter for metadata table
                end_of_data = blanks[blanks > skiprows].min()   # Calculate the end of data table by finding the smallest blank index after the beginning of data table
                read_in_info.append((skiprows, end_of_data - skiprows + 1, skiprows_meta))   # add the skiprows, caculated number of data lines and skiprows for metadata parameters to the list as a tuple
                data_format = 'plate1'

            if item != [] and re.findall(r"Plate information", item[0]) == ['Plate information'] and re.search(r'Results for', all_data_lines[index + 9][0]) != None:
                skiprows = index + 10
                skiprows_meta = index + 1
                end_of_data = blanks[blanks > skiprows].min()
                read_in_info.append((skiprows, end_of_data - skiprows, skiprows_meta))
                data_format = 'plate2'

            if item != [] and len(item) > 1 and re.findall(r"G-factor", item[0]) == ["G-factor"]:
                g_factor = float(item[4])   
        
        return FA.pre_process_plate(csv_file, read_in_info, data_format, wells), g_factor #csv_file, read_in_info, g_factor, data_format, wells

    
    def pre_process_plate(csv_file, read_in_info, data_format, wells):    

        """ Iterates over the raw data file and creates data frames for the data and metadata for each channel, converts them into
        a 384 or 96 by 1 format and adds them into a dictionary.

        :param csv file: Raw data file in csv format.
        :param type: str
        :param read_in_info: A list containg tuples with read in parameters for each channel.
        :param type: list
        :param data_format: Type of plate (plate1 or plate2).
        :param type: str
        :param wells: Number of wells on the plate.
        :param type: int
        :return: A dictionary containg a dictionary for each repeat containg the metadata df and a dictionary with s and p channel dfs. """ 
        
        row_letters = list(string.ascii_uppercase)[0:plate_dimensions[str(wells)][0]]   # generate a list of letters for well IDs
        col_numbers = list(np.arange(1, plate_dimensions[str(wells)][1]+1).astype(str))   # generate a list of numbers for well IDs
        well_ids = ['%s%s' % (item[0], item[1]) for item in product(row_letters, col_numbers)]   # generate a list of well IDs for the data table

        data_frames = {}   # dictionary to store the data frames
        counter = 1   # counter incremented by 0.5 to enable alternating labelling of data frames as p or s

        for index, item in enumerate(read_in_info):   # iterate over each tuple in the list

            if data_format == 'plate1':   # raw data table does not have row and column names so 'names' parameter passed to omit the last column
                raw_data = pd.read_csv(csv_file, sep=',', names=col_numbers, index_col=False, engine='python', skiprows=item[0], nrows=item[1], encoding='utf-8')

            if data_format == 'plate2':   # raw data table has row an column names, so 'index_col' is 0 to set the first column as row labels
                raw_data = pd.read_csv(csv_file, sep=',', index_col=0, engine='python', skiprows=item[0], nrows=item[1], encoding='utf-8')
                raw_data.drop(raw_data.columns[-1], axis=1, inplace=True)   # delete the last column because it is empty

            # generate df for metadata (number of rows is always 1) and convert measurement time into datetime object   
            metadata = pd.read_csv(csv_file, sep=',', engine='python', skiprows=item[2], nrows=1, encoding='utf-8').astype({'Measurement date': 'datetime64[ns]'})
            data_as_array = np.reshape(raw_data.to_numpy(), (int(wells), 1))   # convert data frames to numpy arrays and reshape into 1D array

            if counter % 1 == 0: 
                new_data = pd.DataFrame(data=data_as_array, index=well_ids, columns=['p'])   # generate new 384 (or 96) by 1 data frame with p channel data
                data_frames[f'repeat_{int(counter)}'] = {'metadata':metadata, 'data': {'p': new_data, 's':''}}   # add data and metadata dfs to the dictionary

            if counter % 1 != 0:
                new_data = pd.DataFrame(data=data_as_array, index=well_ids, columns=['s'])   # generate new 384 (or 96) by 1 data frame with s channel data
                data_frames[f'repeat_{int(counter-0.5)}']['data']['s'] = new_data

            counter = counter + 0.5
        
        return data_frames


    def read_in_list(csv_file, wells):

        """ Iterates over the csv file to find  the line numbers at which the metadata table and raw data table
        begin. Creates two pandas data frames: for the raw data and metadata.

        :param csv_file: Raw data file in csv format.
        :param type: str
        :param wells: Number of wells on the plate.
        :param type: int
        :return: A tuple with raw data and metadata dfs."""

        with open(csv_file) as file:  
            all_data_lines = list(csv.reader(file, delimiter=',')) # read the csv file and cast it into a list containing all lines

        # set the skiprows to be initially greater than the total number of lines in the files to enable the evaluation of if statement until the 'skiprows' parameter is found
        skiprows = len(all_data_lines) + 1 
        # list containing indices of all blank rows
        blank_indices = list(index for index, item in enumerate(all_data_lines) if item == [])   # list containing indices of all blank rows
        blanks = np.array(blank_indices)   # convert the list of blank indices to a numpy array

        # iterate over all lines to find beggining of the data table ('skiprows') and determine the format of data  (list A, B, or C)
        for index, item in enumerate(all_data_lines):   
            if item != [] and len(item) == 1 and re.findall(r"Plate information", item[0]) == ["Plate information"]:
                skiprows_meta = index + 1
                end_of_metadata = blanks[blanks > skiprows_meta].min()   # find the end of metadata by finding the smallest blank index after the beginning of metadata
                nrows_meta = end_of_metadata - skiprows_meta - 1   # calucalte the length of metadata table (the number f rows depends on the number of repeats)

            if item != [] and len(item) >= 2 and re.findall(r"PlateNumber", item[0]) == ['PlateNumber'] and re.findall(r"PlateRepeat", item[1]) == ['PlateRepeat']:   # find line number with the beggining of the data
                skiprows = index - 1
                data_format = 'listA'
                end_of_data = blanks[blanks > skiprows].min()

            if item != [] and len(item) >= 2 and re.findall(r"Plate", item[0]) == ['Plate'] and re.findall(r"Barcode", item[1]) == ['Barcode']:   # find line number with the beggining of the data
                skiprows = index
                data_format = 'listB'
                end_of_data = blanks[blanks > skiprows].min()

            if item != [] and len(item) >= 2 and re.findall(r"Plate", item[0]) == ['Plate']  and re.findall(r"Well", item[1]) == ['Well']:
                skiprows = index
                data_format = 'listC'
                end_of_data = blanks[blanks > skiprows].min()

            if item != [] and re.findall(r"G-factor", item[0]) == ["G-factor"]:   # find the g factor
                g_factor = float(item[4])

        nrows = end_of_data - skiprows - 1   # calculate the length of data table

        raw_data = pd.read_csv(csv_file, sep=',', engine='python', skiprows=skiprows, nrows=nrows, encoding='utf-8')
        raw_metadata = pd.read_csv(csv_file, sep=',', engine='python', skiprows=skiprows_meta, nrows=nrows_meta, encoding='utf-8')

        return FA.pre_process_list(raw_data, raw_metadata, data_format, wells), g_factor


    def pre_process_list(raw_data, raw_metadata, data_format, wells):

        """Extracts the data for each channel and repeat from the raw data table and adds to a dictionary.

        :param  read_in_info: A tuple containing data frames for raw data and metadata.string representing type of list (A, B, or C).
        :param type: tuple
        :param data_format: Type of list (A, B, or C).
        :param type: str
        :return: A dictionary containg a dictionary for each repeat containg the metadata df and a dictionary with s and p channel dfs."""

        row_letters = list(string.ascii_uppercase)[0:plate_dimensions[str(wells)][0]]   # generate a list of letters for well IDs
        col_numbers = list(np.arange(1, plate_dimensions[str(wells)][1]+1).astype(str))   # generate a list of numbers for well IDs
        well_ids = ['%s%s' % (item[0], item[1]) for item in product(row_letters, col_numbers)]   # generate a list of well IDs for the data table

        data_frames = {}   # dictionary to store data frames
        repeats = list(raw_metadata['Repeat'].to_numpy())   # generate a list with repeats based on the metadata table, e.g. for 3 repeats -> [1,2,3]

        # remove the '0' from middle position of well numbers (A01 -> A1), done by reassigning the 'Well' column to a Series containing modified well numbers
        raw_data['Well'] = raw_data['Well'].apply(lambda x: x[0] + x[2] if x[1] == '0' else x)

        for index, repeat in enumerate(repeats):   # iterate over the number of repeats
            if data_format == 'listA':
                groupped_data = raw_data.groupby(raw_data.PlateRepeat).get_group(repeat)   # group and extract the data by the plate repeat column, i.e. in each iteration get data only for the current repeat 

                p_groupped = groupped_data.iloc[::3, :]   # extract data only for the p channel, i.e. each third row starting from the first row
                s_groupped = groupped_data.iloc[1::3, :]   # extract data only for the s channel, i.e. each third row starting from the second row

                p_raw = p_groupped[['Well', 'Signal']]   # extract only the two relevant columns
                s_raw = s_groupped[['Well', 'Signal']]   # for each channel

            if data_format in ['listB', 'listC']: 
                # the column naming is different for the first repeat, i.e. just 'Signal', then it's 'Signal.1', 'Signal.2', etc.
                if repeat == 1: 
                    p_raw = raw_data[['Well', 'Signal']]   
                    s_raw = raw_data[['Well', f'Signal.{repeat}']]
                else:
                    p_raw = raw_data[['Well', f'Signal.{repeat + index - 1}']]   # the column to be extracted is calculated in each iteration
                    s_raw = raw_data[['Well', f'Signal.{repeat + index}']]

            raw_frame = pd.DataFrame(well_ids, columns=['Wells'])
            frame = raw_frame.set_index('Wells')
            
            # set row indices as the well numbers and rename the 'Signal' column to 'p' or 's'
            p_raw.set_index('Well', inplace=True)
            p_raw.set_axis(['p'], axis=1, inplace=True)
            p_channel = frame.join(p_raw)
            s_raw.set_index('Well', inplace=True)
            s_raw.set_axis(['s'], axis=1, inplace=True)
            s_channel = frame.join(s_raw)
    
            metadata = raw_metadata.iloc[[repeat-1]].astype({'Measurement date': 'datetime64[ns]'})   # extract the row with metadata relevant for each repeat and covert date and time into a datetime object
            data_frames[f'repeat_{repeat}'] = {'metadata': metadata, 'data': {'p': p_channel, 's': s_channel}}   # add data frames to the dictionary

        return data_frames

In [16]:
my_data = FA.read_in_envision(csv_file=list_A_repeat, data_type='list', wells=384)

In [17]:
my_data.g_factor

1.15

In [19]:
my_data.data_dict['repeat_1']['data']['s']

,s
Wells,
A1,20469296.0
A2,29296716.0
A3,18210982.0
A4,23159988.0
A5,24960618.0
...,...
P20,NaN
P21,NaN
P22,NaN


In [20]:
my_data2 = FA.read_in_envision(csv_file=list_B_repeat_96, data_type='list', wells=96)

In [21]:
my_data2.g_factor

1.0

In [22]:
my_data2.data_dict['repeat_1']['metadata']

,Plate,Repeat,Barcode,Measured height,Chamber temperature at start,Chamber temperature at end,Humidity at start,Humidity at end,Ambient temperature at start,Ambient temperature at end,Measurement date,Unnamed: 11
0,1,1,NaN,14.52,18.78,18.68,61.8,62.3,18.88,18.98,2020-11-17 13:39:13,NaN


In [24]:
my_data2.data_dict['repeat_2']['data']['p'].iloc[0:50]

,p
Wells,
A1,352962.0
A2,357763.0
A3,541382.0
A4,NaN
A5,NaN
A6,NaN
A7,NaN
A8,NaN
A9,NaN
